In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv 
from sqlalchemy import create_engine
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys 
sys.path.append('../maths/') 
from baseball_stats import BasicFielding

In [ ]:
k = ['playerID', 'birthYear', 'birthMonth', 'birthDay']
import_people = pd.read_csv('../datafiles/People.csv', encoding='latin-1', usecols=k)
import_fielding = pd.read_csv('../datafiles/Fielding.csv', encoding='latin-1')
people = import_people.copy()

#fix fucky dates
people['birthYear'] = people['birthYear'].fillna(1875).astype(int)
people['birthMonth'] = people['birthMonth'].fillna(1).astype(int)
people['birthDay'] = people['birthDay'].fillna(1).astype(int)

#create a birthdate column so we can calulate an age at start of season
people['birthdate'] = people.apply(lambda x: f"""{x['birthYear']}-{x['birthMonth']}-{x['birthDay']}""", axis=1)
people['birthdate'] = pd.to_datetime(people['birthdate'], errors='coerce')

#merge the people and batting dataframes
fielding = people.copy().merge(import_fielding, on='playerID', how ='inner')
fielding['season_start'] = fielding.apply(lambda x: f"""{x['yearID']}-04-01""", axis=1)
fielding['season_start'] = pd.to_datetime(fielding['season_start'], errors='coerce')
fielding['age'] = fielding.apply(lambda x: relativedelta(x['season_start'], x['birthdate']).years, axis=1)

fielding = fielding.drop(columns = ['birthYear', 'birthMonth', 'birthdate', 'birthDay', 'season_start', 'stint', 'teamID', 'lgID', 'ZR', 'POS'])
fielding.rename(columns = {'yearID':'Years', 'InnOuts':'IPO', 'SB':'SBA', 'CS':'CCS'}, inplace=True)

context_cols = ['playerID', 'age', 'Years']
data_cols = fielding.columns.difference(context_cols)

fielding[data_cols] = fielding[data_cols].fillna(0)
fielding = fielding[context_cols + data_cols.tolist()]

fielding = fielding.groupby(['playerID', 'age', 'Years']).sum().reset_index()

#easy adding shit up
fielding = BasicFielding.fieldingSums(fielding)

,playerID,age,Years,A,CCS,DP,E,G,GS,IPO,PB,PO,SBA,WP,IP,TC,CA,SBAT
0,aardsda01,22,2004,0,0.0,0,0.0,11,0.0,32.0,0.0,0,0.0,0.0,10.667,0.0,0,0.0
1,aardsda01,24,2006,5,0.0,1,0.0,45,0.0,159.0,0.0,1,0.0,0.0,53.000,6.0,6,0.0
2,aardsda01,25,2007,4,0.0,0,1.0,25,0.0,97.0,0.0,2,0.0,0.0,32.333,7.0,6,0.0
3,aardsda01,26,2008,6,0.0,0,0.0,47,0.0,146.0,0.0,3,0.0,0.0,48.667,9.0,9,0.0
4,aardsda01,27,2009,5,0.0,1,0.0,73,0.0,214.0,0.0,2,0.0,0.0,71.333,7.0,7,0.0
5,aardsda01,28,2010,3,0.0,0,1.0,53,0.0,149.0,0.0,2,0.0,0.0,49.667,6.0,5,0.0
6,aardsda01,30,2012,0,0.0,0,0.0,1,0.0,3.0,0.0,0,0.0,0.0,1.000,0.0,0,0.0
7,aardsda01,31,2013,5,0.0,0,0.0,43,0.0,119.0,0.0,1,0.0,0.0,39.667,6.0,6,0.0
8,aardsda01,33,2015,1,0.0,0,1.0,33,0.0,92.0,0.0,0,0.0,0.0,30.667,2.0,1,0.0
9,aaronha01,20,1954,5,0.0,0,7.0,116,113.0,3093.0,0.0,223,0.0,0.0,1031.000,235.0,228,0.0


In [15]:
career_df = fielding.copy()
career_df = career_df.groupby(['playerID']).agg({
    'age':'mean',
    'Years':'count',
    'A':'sum',
    'CCS':'sum',
    'DP':'sum',
    'E':'sum',
    'G':'sum',
    'GS':'sum',
    'IPO':'sum',
    'PB':'sum',
    'PO':'sum',
    'IPO':'sum',
    'SBA':'sum',
    'WP':'sum',
    'IP':'sum',
    'TC':'sum',
    'CA':'sum',
    'SBAT':'sum'
}).reset_index()

career_df['age'] = career_df['age'].round(0).astype(int)

In [16]:
avg162_df = career_df.copy()

stat_cols = avg162_df.columns.difference(['playerID', 'age', 'Years'])
avg162_df[stat_cols] = avg162_df[avg162_df['G'] > 0][stat_cols].div(avg162_df['G'], axis=0).mul(162).round(2)
avg162_df.insert(0, 'rowType', '162Avg')


In [17]:
norm_df = career_df.copy()
stat_cols = norm_df.columns.difference(['playerID', 'age', 'Years'])
norm_df[stat_cols] = norm_df[norm_df['IP'] > 0][stat_cols].div(norm_df['IP'], axis=0).mul(9).round(2)
norm_df.insert(0, 'rowType', 'normalized')

In [ ]:
#merge all but season average and caluclate the percentages and ratios

season_df = fielding.copy()
season_df.insert(0, 'rowType', 'season')
career_df.insert(0, 'rowType', 'career')

most_fielding = pd.concat([season_df, career_df, avg162_df, norm_df], ignore_index=True)

most_fielding = BasicFielding.fieldingRatios(most_fielding)

,rowType,playerID,age,Years,A,CCS,DP,E,G,GS,...,WP,IP,TC,CA,SBAT,Fp,RF,RF9,Sp,SBp
0,season,aardsda01,22,2004,0.0,0.0,0.0,0.0,11.0,0.0,...,0.0,10.667,0.0,0.0,0.0,NaN,0.000,0.000,0.000,NaN
1,season,aardsda01,24,2006,5.0,0.0,1.0,0.0,45.0,0.0,...,0.0,53.000,6.0,6.0,0.0,1.000,0.133,1.019,0.000,NaN
2,season,aardsda01,25,2007,4.0,0.0,0.0,1.0,25.0,0.0,...,0.0,32.333,7.0,6.0,0.0,0.857,0.240,1.670,0.000,NaN
3,season,aardsda01,26,2008,6.0,0.0,0.0,0.0,47.0,0.0,...,0.0,48.667,9.0,9.0,0.0,1.000,0.191,1.664,0.000,NaN
4,season,aardsda01,27,2009,5.0,0.0,1.0,0.0,73.0,0.0,...,0.0,71.333,7.0,7.0,0.0,1.000,0.096,0.883,0.000,NaN
5,season,aardsda01,28,2010,3.0,0.0,0.0,1.0,53.0,0.0,...,0.0,49.667,6.0,5.0,0.0,0.833,0.094,0.906,0.000,NaN
6,season,aardsda01,30,2012,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.000,0.0,0.0,0.0,NaN,0.000,0.000,0.000,NaN
7,season,aardsda01,31,2013,5.0,0.0,0.0,0.0,43.0,0.0,...,0.0,39.667,6.0,6.0,0.0,1.000,0.140,1.361,0.000,NaN
8,season,aardsda01,33,2015,1.0,0.0,0.0,1.0,33.0,0.0,...,0.0,30.667,2.0,1.0,0.0,0.500,0.030,0.293,0.000,NaN
9,season,aaronha01,20,1954,5.0,0.0,0.0,7.0,116.0,113.0,...,0.0,1031.000,235.0,228.0,0.0,0.970,1.966,1.990,0.974,NaN


In [ ]:
#create rows for season average
season_avg_df = most_fielding[most_fielding['rowType'] == 'season'].copy()
season_avg_df = season_avg_df.drop(['rowType'], axis=1).groupby(['playerID']).mean().round(4).reset_index()

season_avg_df['age'] = season_avg_df['age'].round(0).astype(int)
season_avg_df['Years'] = season_avg_df['Years'].round(0).astype(int)
season_avg_df.insert(0, 'rowType', 'seasAvg')

final_fielding = pd.concat([most_fielding, season_avg_df], ignore_index=True)

,rowType,playerID,age,Years,A,CCS,DP,E,G,GS,...,WP,IP,TC,CA,SBAT,Fp,RF,RF9,Sp,SBp
73847,season,piazzmi01,23,1992,7.00,4.00,1.0000,1.00,16.0000,16.0000,...,0.0,139.3330,102.0000,101.0000,14.0000,0.990,6.3120,6.5240,1.0000,0.2860
73848,season,piazzmi01,24,1993,99.00,59.00,11.0000,11.00,147.0000,141.0000,...,0.0,1244.3330,1011.0000,1000.0000,167.0000,0.989,6.8030,7.2330,0.9590,0.3530
73849,season,piazzmi01,25,1994,38.00,26.00,7.0000,10.00,104.0000,99.0000,...,0.0,860.6670,688.0000,678.0000,102.0000,0.985,6.5190,7.0900,0.9520,0.2550
73850,season,piazzmi01,26,1995,52.00,29.00,8.0000,9.00,112.0000,110.0000,...,0.0,941.0000,866.0000,857.0000,117.0000,0.990,7.6520,8.1970,0.9820,0.2480
73851,season,piazzmi01,27,1996,70.00,34.00,6.0000,9.00,146.0000,144.0000,...,0.0,1255.6670,1134.0000,1125.0000,189.0000,0.992,7.7050,8.0630,0.9860,0.1800
73852,season,piazzmi01,28,1997,74.00,43.00,10.0000,16.00,139.0000,139.0000,...,0.0,1200.0000,1135.0000,1119.0000,155.0000,0.986,8.0500,8.3920,1.0000,0.2770
73853,season,piazzmi01,29,1998,85.00,41.00,7.0000,11.00,140.0000,140.0000,...,0.0,1190.0000,1080.0000,1069.0000,156.0000,0.990,7.6360,8.0850,1.0000,0.2630
73854,season,piazzmi01,30,1999,47.00,37.00,5.0000,11.00,137.0000,135.0000,...,0.0,1156.6670,1011.0000,1000.0000,152.0000,0.989,7.2990,7.7810,0.9850,0.2430
73855,season,piazzmi01,31,2000,38.00,32.00,10.0000,3.00,124.0000,124.0000,...,0.0,1027.6670,903.0000,900.0000,142.0000,0.997,7.2580,7.8820,1.0000,0.2250
73856,season,piazzmi01,32,2001,58.00,33.00,5.0000,9.00,131.0000,128.0000,...,0.0,1085.3330,986.0000,977.0000,147.0000,0.991,7.4580,8.1020,0.9770,0.2240


In [ ]:
load_dotenv()

db_user=os.getenv('jbbs_db_user')
db_pass=os.getenv('jbbs_db_password')
db_host=os.getenv('jbbs_db_host')
db_name=os.getenv('jbbs_db_name')

engine = create_engine(
    f'mysql+mysqlconnector://{db_user}:{db_pass}@{db_host}/{db_name}',
    echo=False,
    pool_size=5,      # Maximum number of connections in the pool
    max_overflow=0,   # Prevents creating more connections than `pool_size`
    pool_recycle=600,
    pool_pre_ping=True
)

with engine.connect() as conn:
    conn.rollback() 
    final_fielding.to_sql('player_fielding_new', con=engine, if_exists='replace', index=False, chunksize=500)
    conn.execute('create index idx_playerid on player_fielding_new(playerID);')
    conn.execute('create index idx_rowType on player_fielding_new(rowType);')
    conn.execute('create index idx_age on player_fielding_new(age);')
    conn.execute('create index idx_years on player_fielding_new(Years);')
